In [2]:
from ROOT import TFile
from ROOT import TCanvas, TPad, TLegend, TLatex

Welcome to JupyROOT 6.24/06


In [11]:
configs = {
    #"2016preVFP/Electron/eff_electron": {
    #    "eff_range": [0.4, 1.1],
    #    "sf_range": [0.9, 1.1],
    #    "eta_bins": [0, 0.8, 1.444, 1.566, 2, 2.5],
    #    "info": "L^{int} = 19.8 /fb"
    #},
    #"2016postVFP/Electron/eff_electron": {
    #    "eff_range": [0.4, 1.1],
    #    "sf_range": [0.9, 1.1],
    #    "eta_bins": [0, 0.8, 1.444, 1.566, 2, 2.5],
    #    "info": "L^{int} = 19.8 /fb"
    #},
    #"2017/Electron/eff_electron": {
    #    "eff_range": [0.4, 1.1],
    #    "eta_bins": [0, 0.8, 1.444, 1.566, 2, 2.5],
    #    "info": "L^{int} = 41.5 /fb"
    #},
    #"2018/Electron/eff_electron": {
    #    "eff_range": [0.4, 1.1],
    #    "sf_range": [0.8, 1.1],
    #    "eta_bins": [0, 0.8, 1.444, 1.566, 2, 2.5],
    #    "info": "L^{int} = 59.7 /fb"
    #},
    "2016preVFP/Muon/Effi_POGMedium-dZ01-SIP4-miniIso01": {
        "eff_range": [0.4, 1.1],
        "sf_range": [0.9, 1.1],
        "eta_bins": [0, 0.9, 1.2, 2.1, 2.4],
        "info": "L^{int} = 19.8 /fb"
    },
    "2016postVFP/Muon/Effi_POGMedium-dZ01-SIP4-miniIso01": {
        "eff_range": [0.4, 1.1],
        "sf_range": [0.9, 1.1],
        "eta_bins": [0, 0.9, 1.2, 2.1, 2.4],
        "info": "L^{int} = 19.8 /fb"
    },
    "2017/Muon/Effi_POGMedium-dZ01-SIP4-miniIso01": {
        "eff_range": [0.4, 1.1],
        "sf_range": [0.9, 1.1],
        "eta_bins": [0, 0.9, 1.2, 2.1, 2.4],
        "info": "L^{int} = 41.5 /fb"
    },
    "2018/Muon/Effi_POGMedium-dZ01-SIP4-miniIso01": {
        "eff_range": [0.4, 1.1],
        "sf_range": [0.8, 1.2],
        "eta_bins": [0, 0.9, 1.2, 2.1, 2.4],
        "info": "L^{int} = 59.7 /fb"
    }
}

In [12]:
class Canvas():
    def __init__(self, configs=None):
        self.cvs = TCanvas("c", "", 720, 900)
        self.pad_up = TPad("up", "", 0, 0.25, 1, 1)
        self.pad_up.SetGrid(True)
        self.pad_up.SetBottomMargin(0.02)
        self.pad_down = TPad("down", "", 0, 0, 1, 0.25)
        self.pad_down.SetGrid(True)
        self.pad_down.SetTopMargin(0.08)
        self.pad_down.SetBottomMargin(0.3)
        
        self.legend = TLegend(0.45, 0.05, 0.9, 0.43)
        self.legend.SetFillStyle(0)
        self.legend.SetBorderSize(0)
        self.legend.SetNColumns(2)
        
        self.info = TLatex()
        self.info.SetTextSize(0.03)
        self.info.SetTextFont(42)
        self.logo = TLatex()
        self.logo.SetTextSize(0.04)
        self.logo.SetTextFont(61)
        self.extra_logo = TLatex()
        self.extra_logo.SetTextSize(0.035)
        self.extra_logo.SetTextFont(52)
        
        self.configs = configs
    
    def get_hists(self, eff_data, eff_mc, sf):
        self.eff_data = eff_data
        self.eff_mc = eff_mc
        self.sf = sf
        self.__make_projections()
        self.__decorate()
    
    def __make_projections(self):
        self.proj_eff_data = dict()
        self.proj_eff_mc = dict()
        self.proj_sf = dict()
        
        for i in range(1, len(self.configs['eta_bins'])):
            bin_range = f"{self.configs['eta_bins'][i-1]} < |#eta| < {self.configs['eta_bins'][i]}"
            self.proj_eff_data[bin_range] = self.eff_data.ProjectionY(f"eff_data_{bin_range}", i, i)
            self.proj_eff_mc[bin_range] = self.eff_mc.ProjectionY(f"eff_mc_{bin_range}", i, i)
            self.proj_sf[bin_range] = self.sf.ProjectionY(f"sf_{bin_range}", i, i)
            
        # make systematic histograms
        self.syst_eff_data = dict()
        self.syst_eff_mc = dict()
        self.syst_sf = dict()
        for name in self.proj_eff_data.keys():
            self.syst_eff_data[name] = self.proj_eff_data[name].Clone(f"syst_data_{name}")
            self.syst_eff_mc[name] = self.proj_eff_data[name].Clone(f"syst_data_{name}")
            self.syst_sf[name] = self.proj_sf[name].Clone(f"syst_data_{name}")
    
    def __decorate(self):
        if "eff_range" in self.configs.keys():
            down = self.configs["eff_range"][0]
            up = self.configs["eff_range"][1]
        else:
            down = 0.4
            up = 1.
        
        colors = [1, 2, 8, 4, 6]
        for i, (name, hist) in enumerate(self.proj_eff_data.items()):
            color = colors[i]
            hist.SetStats(0)
            hist.SetTitle("")
            hist.GetXaxis().SetTitleSize(0)
            hist.GetXaxis().SetLabelSize(0)
            hist.GetXaxis().SetRangeUser(10., 200.)
            hist.GetYaxis().SetTitle("efficiency")
            hist.GetYaxis().SetTitleOffset(1.25)
            hist.GetYaxis().SetRangeUser(down, up)
            hist.SetLineColor(color)
            hist.SetMarkerColor(color)
            hist.SetMarkerStyle(21)
            hist.SetMarkerSize(1)
        
        for i, (name, hist) in enumerate(self.syst_eff_data.items()):
            color= colors[i]
            hist.SetStats(0)
            hist.SetTitle("")
            hist.GetXaxis().SetTitleSize(0)
            hist.GetXaxis().SetLabelSize(0)
            hist.GetXaxis().SetRangeUser(10., 200.)
            hist.GetYaxis().SetTitle("efficiency")
            hist.GetYaxis().SetTitleOffset(1.25)
            hist.GetYaxis().SetRangeUser(down, up)
            hist.SetFillColorAlpha(color, 0.7)
            hist.SetFillStyle(3244)
        
        colors = [12, 46, 30, 9, 40]
        for i, (name, hist) in enumerate(self.proj_eff_mc.items()):
            color = colors[i]
            hist.SetStats(0)
            hist.SetTitle("")
            hist.GetXaxis().SetRangeUser(10., 200.)
            hist.GetYaxis().SetRangeUser(down, up)
            hist.SetLineColor(color)
            hist.SetMarkerColor(color)
            hist.SetMarkerStyle(21)
            hist.SetMarkerSize(1)
        
        for i, (name, hist) in enumerate(self.syst_eff_mc.items()):
            color = colors[i]
            hist.SetStats(0)
            hist.SetTitle("")
            hist.GetXaxis().SetRangeUser(10., 200.)
            hist.GetYaxis().SetRangeUser(down, up)
            hist.SetFillColorAlpha(color, 0.7)
            hist.SetFillStyle(3244)
            
        if "sf_range" in self.configs.keys():
            down = self.configs["sf_range"][0]
            up = self.configs["sf_range"][1]
        else:
            down = 0.9 
            up = 1.1
        
        colors = [1, 2, 8, 4, 6]
        for i, (name, hist) in enumerate(self.proj_sf.items()):
            color = colors[i]
            hist.SetStats(0)
            hist.SetTitle("")
            hist.GetXaxis().SetTitle("p_{T} [GeV]")
            hist.GetXaxis().SetTitleSize(0.1)
            hist.GetXaxis().SetTitleOffset(0.9)
            hist.GetXaxis().SetLabelSize(0.08)
            hist.GetXaxis().SetRangeUser(10., 200.)
            hist.GetYaxis().SetTitle("Data / MC")
            hist.GetYaxis().CenterTitle()
            hist.GetYaxis().SetTitleSize(0.1)
            hist.GetYaxis().SetTitleOffset(0.4)
            hist.GetYaxis().SetLabelSize(0.08)
            hist.GetYaxis().SetRangeUser(down, up)
            hist.SetLineColor(color)
            hist.SetMarkerColor(color)
            hist.SetMarkerStyle(20)
            hist.SetMarkerSize(0.6)
    
    def combine(self):
        self.pad_up.cd()
        #for name, hist in self.syst_eff_data.items():
        #    hist.Draw("e2&same")
        #for name, hist in self.syst_eff_mc.items():
        #    hist.Draw("e2&same")
        for name in self.proj_eff_data.keys():
            self.proj_eff_data[name].Draw("p&same")
            self.proj_eff_mc[name].Draw("p&same")
            self.legend.AddEntry(self.proj_eff_data[name], f"DATA {name}", "lep")
            self.legend.AddEntry(self.proj_eff_mc[name], f"MC {name}", "lep")
        self.legend.Draw("same")
        if "info" in self.configs.keys():
            self.info.DrawLatexNDC(0.74, 0.91, self.configs["info"])
        self.logo.DrawLatexNDC(0.15, 0.83, "CMS")
        self.extra_logo.DrawLatexNDC(0.15, 0.78, "Work in progress")
        
        self.pad_down.cd()
        for hist in self.proj_sf.values():
            hist.Draw("p&same")
        
        self.cvs.cd()
        self.pad_up.Draw()
        self.pad_down.Draw()
    
    def draw(self):
        self.cvs.Draw()
    
    def savefig(self, figname):
        self.cvs.SaveAs(figname)

In [14]:
for fkey in configs.keys():
    f = TFile.Open(f"{fkey}.root")
    eff_data = f.Get("data_s0m0") if "Electron" in fkey else f.Get("muonEffi_data_eta_pt")
    eff_mc = f.Get("sim_s0m0") if "Electron" in fkey else f.Get("muonEffi_mc_eta_pt")
    sf = f.Get("sf_s0m0") if "Electron" in fkey else f.Get("SF_eta_pt")

    cvs = Canvas(configs[fkey])
    cvs.get_hists(eff_data, eff_mc, sf)
    cvs.combine()
    cvs.savefig(f"../www/HcToWA/UltraLegacy/MetaInfo/Efficiencies/{fkey.replace('/', '_')}.png")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file ../www/HcToWA/UltraLegacy/MetaInfo/Efficiencies/2016preVFP_Muon_Effi_POGMedium-dZ01-SIP4-miniIso01.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file ../www/HcToWA/UltraLegacy/MetaInfo/Efficiencies/2016postVFP_Muon_Effi_POGMedium-dZ01-SIP4-miniIso01.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file ../www/HcToWA/UltraLegacy/MetaInfo/Efficiencies/2017_Muon_Effi_POGMedium-dZ01-SIP4-miniIso01.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file ../www/HcToWA/UltraLegacy/MetaInfo/Efficiencies/2018_Muon_Effi_POGMedium-dZ01-SIP4-miniIso01.png has been created
